In [4]:
%load_ext autoreload
%autoreload 2

import os
import re

# GET PROJECT'S ROOT DIR
BASE_PATH = re.search(r'.+(Team5)', os.getcwd())[0]
os.chdir(BASE_PATH)
BASE_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/projects/master/c1/Team5'

In [ ]:
"""
# TASK 1: Compute image descriptors (QS1)

Color Histogram:
    - gray level / concatenate color component histograms
    - color space RGB, CieLab, YCbCr, HSV, etc.
    - Compulsory to use 1D histograms!
"""

In [8]:
DATA_DIRECTORY = './data'

In [12]:
# Read data
import zipfile

def extract(path_to_zip_file: str, directory_to_extract_to: str) -> None:
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

extract(f'{DATA_DIRECTORY}/BBDD.zip', DATA_DIRECTORY)
extract(f'{DATA_DIRECTORY}/qsd1_w1.zip', DATA_DIRECTORY)

In [116]:
from enum import Enum
from typing import Optional, List, Callable
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import re

class ColorSpace(Enum):
    gray = cv2.COLOR_BGR2GRAY
    RGB = cv2.COLOR_BGR2RGB
    HSV = cv2.COLOR_BGR2HSV
    CieLab = cv2.COLOR_BGR2Lab
    YCbCr = cv2.COLOR_BGR2YCrCb

class ImageDataset:
    def __init__(self, directory_path: str, colorspace: ColorSpace = ColorSpace.RGB, interval: int = 1):
        self.directory_path = directory_path
        self.colorspace = colorspace
        self.interval = interval
        self.descriptors = self.load_dataset()

    def load_dataset(self):
        """
        Load all '.jpg' images from the specified directory and create Image instances for each.
        Stores the Image instances in self.descriptors.
        """

        # List all images in the directory that have '.jpg' extension
        image_filenames = [f for f in os.listdir(self.directory_path) if f.endswith('.jpg')]

        # Create Image instances for each image and store them in self.descriptors
        result = []
        for image_filename in image_filenames:
            image_path = os.path.join(self.directory_path, image_filename)
            image_instance = Image(image_path, self.colorspace, self.interval)  # Assuming Image class takes the image path as an argument
            result.append(image_instance)
        return result


# Normalization of a histogram
def normalize(hist: List[np.array]) -> List[np.array]:
    """
    Normalizes the given histogram. The sum of all bins in the returned histogram is 1.

    Parameters:
    hist (List[np.array]): Histogram.

    Returns:
    hist (List[np.array]): Normalized histogram.
    """
    
    return hist / np.sum(hist)

# Distance metrics
def euclidean_distance(hist1: List[np.array], hist2: List[np.array]) -> float:
    """
    Euclidean distance is the straight-line distance between two points in Euclidean space.

    Parameters:
    hist1 (List[np.array]): The first histogram.
    hist2 (List[np.array]): The second histogram.

    Returns:
    float: The Euclidean distance between the two histograms.
    """
        
    return np.linalg.norm(hist1 - hist2)

def l1_distance(hist1: List[np.array], hist2: List[np.array]) -> float:
    """
    L1 distance is the sum of the absolute differences between corresponding bins
    in the histograms. It is less sensitive to large differences than Euclidean distance.

    Parameters:
    hist1 (List[np.array]): The first histogram.
    hist2 (List[np.array]): The second histogram.

    Returns:
    float: The L1 (Manhattan) distance.
    """

    return np.sum(np.abs(hist1 - hist2))

def chi2_distance(hist1: List[np.array], hist2: List[np.array]) -> float:
    """
    Chi-squared distance is useful to measure similarity between 2 feature matrices.

    Parameters:
    hist1 (List[np.array]): The first histogram.
    hist2 (List[np.array]): The second histogram.

    Returns:
    float: The Chi-squared distance between the two histograms.
    """

    return 0.5 * np.sum(np.square(hist1 - hist2) / (hist1 + hist2 + 1e-10))

def hellinger_kernel_distance(hist1: List[np.array], hist2: List[np.array]) -> float:
    """
    Hellinger distance is useful to quantify the similarity between two probability distributions.

    Parameters:
    hist1 (List[np.array]): The first histogram.
    hist2 (List[np.array]): The second histogram.

    Returns:
    float: The Hellinger kernel distance.
    """
    return np.sqrt(0.5 * np.sum((np.sqrt(hist1) - np.sqrt(hist2)) ** 2))


class DistanceType(Enum):
    euclidean = euclidean_distance
    l1 = l1_distance
    chi2 = chi2_distance


# Similarity metrics
def hellinger_kernel_similarity(hist1: List[np.array], hist2: List[np.array]) -> float:
    """
    Hellinger kernel to quantify the similarity between two probability distributions (or histograms).

    Parameters:
    hist1 (List[np.array]): The first histogram.
    hist2 (List[np.array]): The second histogram.

    Returns:
    float: The Hellinger kernel similarity.
    """
    
    return np.sum(np.sqrt(hist1 * hist2))

def histogram_intersection_similarity(hist1: List[np.array], hist2: List[np.array]) -> float:
    """
    Computes the histogram intersection similarity between two histograms.

    Parameters:
    hist1 (List[np.array]): The first histogram.
    hist2 (List[np.array]): The second histogram.

    Returns:
    float: The histogram intersection similarity (between 0 and 1).
    """
    
    return np.sum(np.minimum(hist1, hist2))

class SimilarityType(Enum):
    hellinger_kernel = hellinger_kernel_similarity
    histogram_intersection = histogram_intersection_similarity
    
    
class Image:
    def __init__(self, path: str, colorspace: ColorSpace = ColorSpace.RGB, interval: int = 1):
        self.path = path
        self.index = self._extract_index(path)
        self.image = cv2.imread(path)
        self.colorspace = colorspace
        self.interval = interval
        self.histogram_descriptor = self.compute_image_histogram_descriptor()


    def _extract_index(self, file_path):
        file_name = file_path.split('/')[-1]
        name = file_name.split('.')[0]
        number = name.split('_')[-1]
        return int(number)


    def compute_image_histogram_descriptor(self):
        # Convert image to colorspace
        converted_img = cv2.cvtColor(self.image, self.colorspace.value)
        # Separate the channels
        channels = cv2.split(converted_img)

        # Create histogram
        histograms = []
        for channel in channels:
            # Compute histogram
            hist, _ = np.histogram(channel, bins=np.arange(0, 256, self.interval))  # Intervals of histogram given by bin_size
            
           # Normalize and flatten histograms
            hist = hist.flatten()
            hist = normalize(hist)
            histograms.append(hist)

        return histograms
    

    def plot_histograms(self, savepath: Optional[str] = None):
        channel_names = self.get_channel_names()

        fig, axs = plt.subplots(1, len(self.histogram_descriptor), figsize=(15, 5), sharey=True)

        for i, hist in enumerate(self.histogram_descriptor):
            axs[i].bar(range(len(hist)), hist, width=0.5, color='blue', alpha=0.7)
            axs[i].set_title(f'{channel_names[i]}')
            axs[i].set_xlabel('Intensity')
            axs[i].set_ylabel('Frequency')
            axs[i].set_xlim(-1, len(hist))
            # axs[i].set_xticks(range(len(hist)))
            # axs[i].set_xticklabels([f"{(j + 1) * self.interval - 1}" for j in range(len(hist))], rotation=45)
            axs[i].grid(False)

        plt.tight_layout()
        fig.suptitle(self.colorspace.name)

        # Save plot if savepath is provided
        if savepath:
            plt.savefig(f"{savepath}/{channel_names[i]}_{self.colorspace.name}_histogram.png")

        plt.show()

    def get_channel_names(self):
        # Asociate the colorspace to the names of their channels
        colorspace_dict = {
            'gray': ['Intensity'],
            'RGB': ['R', 'G', 'B'],
            'HSV': ['H', 'S', 'V'],
            'CieLab': ['L', 'a', 'b'],
            'YCbCr': ['Y', 'Cb', 'Cr']
        }
        return colorspace_dict[self.colorspace.name]

    def show(self):
        rgb_image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        plt.imshow(rgb_image)
        plt.show()
    

    def _compute_similarity_or_distance(self, image2: 'Image', func: Callable) -> List[float]:
        result = []
        
        # Assert they have comparable histograms
        assert self.colorspace.name == image2.colorspace.name
        assert self.interval == image2.interval
        
        for i, _ in enumerate(self.histogram_descriptor):

            # Compute distance/similarity
            result.append(
                func(self.histogram_descriptor[i], image2.histogram_descriptor[i])
            )

        return result

    def compute_similarity(self, image2: 'Image', type=SimilarityType):
        return self._compute_similarity_or_distance(image2, type)

    def compute_distance(self, image2: 'Image', type=DistanceType) -> List[float]:
        return self._compute_similarity_or_distance(image2, type)

In [117]:
BDDataset = ImageDataset(f'{DATA_DIRECTORY}/BBDD', colorspace=ColorSpace.HSV)
QueryDataset = ImageDataset(f'{DATA_DIRECTORY}/qsd1_w1', colorspace=ColorSpace.HSV)

bbdd_00161.jpg
bbdd_00161
bbdd_00175.jpg
bbdd_00175
bbdd_00149.jpg
bbdd_00149
bbdd_00217.jpg
bbdd_00217
bbdd_00203.jpg
bbdd_00203
bbdd_00015.jpg
bbdd_00015
bbdd_00001.jpg
bbdd_00001
bbdd_00029.jpg
bbdd_00029
bbdd_00028.jpg
bbdd_00028
bbdd_00000.jpg
bbdd_00000
bbdd_00014.jpg
bbdd_00014
bbdd_00202.jpg
bbdd_00202
bbdd_00216.jpg
bbdd_00216
bbdd_00148.jpg
bbdd_00148
bbdd_00174.jpg
bbdd_00174
bbdd_00160.jpg
bbdd_00160
bbdd_00189.jpg
bbdd_00189
bbdd_00176.jpg
bbdd_00176
bbdd_00162.jpg
bbdd_00162
bbdd_00200.jpg
bbdd_00200
bbdd_00214.jpg
bbdd_00214
bbdd_00228.jpg
bbdd_00228
bbdd_00002.jpg
bbdd_00002
bbdd_00016.jpg
bbdd_00016
bbdd_00017.jpg
bbdd_00017
bbdd_00003.jpg
bbdd_00003
bbdd_00229.jpg
bbdd_00229
bbdd_00215.jpg
bbdd_00215
bbdd_00201.jpg
bbdd_00201
bbdd_00163.jpg
bbdd_00163


KeyboardInterrupt: 

In [13]:
from src.image_dataset import ImageDataset, ColorSpace

BDDataset = ImageDataset(f'{DATA_DIRECTORY}/BBDD', colorspace=ColorSpace.HSV)
QueryDataset = ImageDataset(f'{DATA_DIRECTORY}/qsd1_w1', colorspace=ColorSpace.HSV)

In [ ]:
QueryDataset.descriptors[1].index

In [94]:
k = 5

result = [[] for _ in range(len(QueryDataset.descriptors))]

for image in QueryDataset.descriptors:
    distances_list = []
    for image2 in BDDataset.descriptors:
        distances = image.compute_distance(image2, type=DistanceType.l1)
        distance = np.mean(distances) # We can play with min, max, sum ...
        distances_list.append(distance)
    top_k = np.argsort(distances_list)[:k]

    result[image.index] = [BDDataset.descriptors[i].index for i in top_k]

## Task 3

In [71]:
# Metrics copied from https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py  

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average precision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [93]:
import pickle

# Load ground truth correspondences
with open(f'{DATA_DIRECTORY}/qsd1_w1/gt_corresps.pkl', 'rb') as f:
    gt = pickle.load(f)

# Call mAP@K for k=1 and k=5
mapk(gt, result, k=5)

In [90]:
image1 = Image(f'{DATA_DIRECTORY}/qsd1_w1/00000.jpg', colorspace=ColorSpace.HSV) # Query
image2 = Image(f'{DATA_DIRECTORY}/BBDD/bbdd_00120.jpg', colorspace=ColorSpace.HSV) # BBDD 120
image3 = Image(f'{DATA_DIRECTORY}/BBDD/bbdd_00229.jpg', colorspace=ColorSpace.HSV) # BBDD 211

print(np.mean(image1.compute_distance(image2, type=DistanceType.l1)))
print(np.mean(image1.compute_distance(image3, type=DistanceType.l1)))


In [87]:
image1.plot_histograms()
image2.plot_histograms()
image3.plot_histograms()